# (YOLO ver. 5) Traffic Sign Recognition

# The data is taken from the German Traffic Sign Recognition Benchmark (GTSRB) Dataset, which has 43 classes and around 50k photos. 

In [ ]:
import pandas as pd
import shutil
import os
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
%cd gdrive/My Drive/Colab Notebooks/traffic

## Darknet Cloning Process

In [ ]:
!git clone 'https://github.com/AlexeyAB/darknet.git'

## Making Adjustments to the Makefile to Use Some of the GPU

In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
!/usr/local/cuda/bin/nvcc --version

## To generate executable applications and libraries from source code, use the make command.

In [ ]:
!make

In [ ]:
!./darknet

In [ ]:
%cd ../../../..

In [ ]:
os_dir ='gdrive/My Drive/Colab Notebooks/traffic/GTSRB/Final_Training/Images/'

### Changing the names of the picture files and their values in the csv file. (This is to keep all of the images in the same folder for processing and to avoid naming disputes.) 

In [ ]:
# Dangerous! Please be careful while editing the codes here.

for dir in os.listdir(os_dir):
    if dir == '.DS_Store' :
        continue
    inner_dir = os.path.join(os_dir, dir)
    print(inner_dir[-5:])
    for img in os.listdir(inner_dir):
        if img == "GT-" + dir + '.csv':
            csv_file = pd.read_csv(os.path.join(inner_dir,"GT-" + dir + '.csv'), sep=';')
            csv_file['Filename'] = csv_file['Filename'].apply(lambda x: f'{inner_dir[-5:]}_{x}')
            csv_file.to_csv(f'{inner_dir}/GGT-{dir}.csv', sep=';',index = False)
        else:
            continue
            os.rename(inner_dir + '/' + img, inner_dir + '/' + f'{inner_dir[-5:]}_{img}')

### All photos are copied to a separate folder.

In [ ]:
for dir in os.listdir(os_dir):
    if dir == '.DS_Store':
        continue
    inner_dir = os.path.join(os_dir, dir)
    print(inner_dir[-5:])
    for img in os.listdir(inner_dir):
        if img == "GT-" + dir + '.csv' or img == "GGT-" + dir + '.csv' or img == '00000_GT-00000.gsheet': # omit all files except the images
            continue
        else:
            shutil.copy(f'{inner_dir}/{img}','gdrive/My Drive/Colab Notebooks/traffic/TFinal')

## Combining all of the csv files into one.

In [ ]:
train_csv = pd.DataFrame()
for dir in os.listdir(os_dir):
    if dir == '.DS_Store' :
        continue
    inner_dir = os.path.join(os_dir, dir)
    csv_file = pd.read_csv(os.path.join(inner_dir,"GGT-" + dir + '.csv'), sep=';')

    train_csv = train_csv.append(csv_file)
print(train_csv.shape)

## For each image, an annotation file is created.

### <--YOLO Format :- [object-class] [x]/[image_width] [y]/[image_height] [width]/[image_width] [height]/[image_height]-->

In [ ]:
train_csv.head()

In [ ]:
IMG_PATH = 'gdrive/My Drive/Colab Notebooks/traffic/TFinal'

In [ ]:
%cd gdrive/My Drive/Colab Notebooks/traffic/TFinal

In [ ]:
def compare(v1, v2):
    if v1 > v2:
        vmax, vmin = v1, v2
        return vmax, vmin
    else:
        vmax, vmin = v2, v1
        return vmax, vmin
def convert_labels(z):
    x1 = z['Roi.X1']
    y1 = z['Roi.Y1']
    x2 = z['Roi.X2']
    y2 = z['Roi.Y2']
    size = [z['Height'],z['Width']]
    xmax, xmin = compare(x1, x2)
    ymax, ymin = compare(y1, y2)
    dw = 1./size[1]
    dh = 1./size[0]
    x = (xmin + xmax)/2.0
    y = (ymin + ymax)/2.0
    w = xmax - xmin
    h = ymax - ymin
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return x,y,w,h

In [ ]:
for index, x in train_csv.iterrows():
    name = x['Filename'].replace('.ppm','.txt')
    labels = convert_labels(x)
    cls = x['ClassId']
    f = open(f'{name}','w+')
    f.write(f'{cls} {labels[0]} {labels[1]} {labels[2]} {labels[3]}')
    f.close()

In [ ]:
!pwd

# Splitting the set into two parts: Training & Testing

In [ ]:
y = train_csv['ClassId'] ## In order to stratify

In [ ]:
from sklearn.model_selection import train_test_split
train_ds, valid_ds = train_test_split(
    train_csv,
    test_size = 0.3,
    random_state = 43,
    shuffle = True,
    stratify = y
)

In [ ]:
train_ds.reset_index()
valid_ds.reset_index()

## Creating txt files with the path to the photos for Training and Testing

In [ ]:
f = open(f'train.txt','w+')
for index, x in train_ds.iterrows():
    name = x['Filename']
    f.write(f'TFinal/{name}\n')
f.close()

In [ ]:
f = open(f'test.txt','w+')
for index, x in valid_ds.iterrows():
    name = x['Filename']
    f.write(f'TFinal/{name}\n')
f.close()

In [ ]:
!pwd

In [ ]:
%cd ..

## Generating labelled data.data and classes.names

In [ ]:
## classes.names ##

classes = [
    'Speed limit (20km/h)',
    'Speed limit (30km/h)',
    'Speed limit (50km/h)',
    'Speed limit (60km/h)',
    'Speed limit (70km/h)',
    'Speed limit (80km/h)',
    'Speed limit (100km/h)',
    'Speed limit (120km/h)',
    'No passing',
    'No passing for vechiles over 3.5 metric tons',
    'No vehicles',
    'Vechiles over 3.5 metric tons prohibited',
    'Right-of-way at the next intersection',
    'General caution',
    'Dangerous curve to the left',
    'Dangerous curve to the right',
    'Double curve',
    'Bumpy road',
    'Slippery road',
    'Road narrows on the right',
    'Road work',
    'Traffic signals',
    'Pedestrians',
    'Children crossing',
    'Bicycles crossing',
    'Beware of ice/snow',
    'Wild animals crossing',
    'Turn right ahead',
    'Turn left ahead',
    'Ahead only',
    'Go straight or right',
    'Go straight or left',
    'Keep right',
    'Keep left',
    'Roundabout mandatory',
    'Priority road',
    'Yield',
    'Stop',
    'No entry',
    'End of speed limit (80km/h)',
    'End of all speed and passing limits',
    'End of no passing',
    'End of no passing by vechiles over 3.5 metric tons'
]

f = open(f'classes.names','w+')
for c in classes:
    f.write(f'{c}\n')
f.close()

## labelled_data.data ##

config = {
    'classes': 43,
    'train': 'TFinal/train.text',
    'valid': 'TFinal/valid.text',
    'names': 'classes.names',
    'backup': 'backup'
}
f = open(f'labelled_data.data', 'w+')
for key in config:
    f.write(f'{key} = {config[key]}\n')
f.close()


# Training the Model

In [ ]:
!wget https://pjreddie.com/media/files/yolov3-tiny.weights ## tinyYOLO for using our prototype car in TUSAŞ (with RaspberryPi)

In [ ]:
!./darknet/darknet detector train labelled_data.data yolov3-tiny_custom.cfg backup/yolov3-tiny_custom_best.weights -dont_show -map